In [ ]:
#%%
import os
import torch
import random
import numpy as np
import argparse
import json
import cohere
from openai import OpenAI


In [ ]:
#%%
from tqdm import tqdm

from collections import Counter

from transformers import LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import hashlib

from textgames import GAME_NAMES, GAME_IDS, LEVELS, LEVELS_HIDDEN, LEVEL_IDS, new_game


In [ ]:
#%%
gen_model_checkpoint = "google/gemma-2-9b-it"
quantize = True

In [ ]:
kwargs = {
    "device_map": "auto",
} if quantize else {}

In [ ]:
#%%
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_checkpoint, **kwargs)
tokenizer = AutoTokenizer.from_pretrained(gen_model_checkpoint, **kwargs)

In [ ]:
gen_model.device

In [ ]:
def get_gemma_response(text):
    # global gen_model, tokenizer
    messages = [
        {"role": "user", "content": text},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(gen_model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = gen_model.generate(
        input_ids,
        max_new_tokens=100,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.2,
        top_p=1
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
text = \
"""
Given a set of rules to calculate point, sort the set of words in decreasing order.
When there 2 or more words with same point, sort lexicographically.

Rules:
- every pair of consecutive consonant gets 5 points
- every pair of consecutive vowel gets 3 points
- add 1 point if there exists exactly 1 'g' in the word
- word less than 5 characters gets 10 points
- word starts with gen gets 100 points
- word ends with ta gets -1000 point

Words:
- genta
- winata
- hudi
- alham
- aji
- ruochen

Print only the answer.
"""

# Answer:
# - aji      10
# - hudi     10
# - ruochen   5  3
# - alham     5
# - genta     5  1  100 -1000
# - winata  -1000

In [ ]:
print(get_gemma_response(text))

In [ ]:
print(get_gemma_response(text))